In [1]:
import numpy as np
import libRead
import pandas as pd
from os import path
#import scipy.stats as st
#import matplotlib.pyplot as plt
import cv2
#from sklearn.cluster import KMeans
#from sklearn.mixture import GaussianMixture
#from sklearn.decomposition import PCA
#import matplotlib.cm as cm

ModuleNotFoundError: No module named 'cv2'

In [10]:
colormaps = ['Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds', 'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu',
             'BuPu', 'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']


if __name__ == '__main__':
    sFolder = '/Users/dmolins/Desktop/david/uni/TFG/Europe/'
    sTypeData = ''
    #sTypeData = 'Shotcharts'
    #yearBeg = 2007
    yearBeg = 2014
    #yearFin = 2013
    yearFin = 2018
    # if path.exists(sFolder + sTypeData + '/' + sTypeData + str(yearBeg) + "-" +  str(yearFin) +  'EL.pkl') == False:
    #     libRead.readShotchartsInd(sFolder + sTypeData + '/', yearBeg, yearFin+1)
    #     #libRead.readShotcharts(sFolder + sTypeData + '/', yearBeg, yearFin+1)
    # shots = pd.read_pickle(sFolder + sTypeData + '/' + sTypeData + str(yearBeg) + "-" +  str(yearFin) +  'EL.pkl')

    if path.exists(sFolder + '/IndShots' + str(yearBeg) + "-" + str(yearFin) + 'EL.pkl') == False:
    #if path.exists(sFolder + sTypeData + '/IndShots' + str(yearBeg) + "-" + str(yearFin) + 'EL.pkl') == False:
        libRead.readShotchartsInd(sFolder + sTypeData + '/', yearBeg, yearFin + 1)
    shots = pd.read_pickle(sFolder + sTypeData + '/IndShots' + str(yearBeg) + "-" + str(yearFin) + 'EL.pkl')

    player = shots['Player'].values

    nRegs = 13


    team = shots['Team'].values
    allTeams = np.unique([item for sublist in team for item in sublist])
    action = shots['IDAction'].values
    allActions = [item for sublist in action for item in sublist]
    shotsX = shots['CoordX'].values
    allX = [item for sublist in shotsX for item in sublist]
    shotsY = shots['CoordY'].values
    allY = [item for sublist in shotsY for item in sublist]
    games = shots['Games'].values

    progX = []
    progY = []
    progSucc = []
    iInShot = -1

    dimW = int(max(allX) - min(allX))
    dimH = int(max(allY) - min(allY))
    matShots = np.zeros((dimW + 1, dimH + 1))
    xmin, xmax = min(allX), max(allX)
    ymin, ymax = min(allY), max(allY)
    facG = 10
    targetTeam = ''

    shotsIn = np.zeros((nRegs+1, len(shotsX)))
    layupsIn = np.zeros((1, len(shotsX)))
    shotsOut = np.zeros((nRegs+1, len(shotsX)))
    layupsOut = np.zeros((1, len(shotsX)))

    actIn = ['2FGM', '3FGM', 'DUNK', 'LAYUPMD']

    if path.exists(sFolder + sTypeData + '/shotsInNew' + str(yearBeg) + '-' + str(yearFin) + targetTeam + '.npy') == False:
        for iReg in range(1, nRegs + 1):
            imReg = np.fliplr(np.transpose(cv2.imread('/Users/dmolins/Desktop/david/uni/TFG/data_shotchart/RegionsCourt/RegionsNew.' + str(iReg).zfill(3) + '.jpeg')[:, :, 0]))
            #imReg = np.fliplr(np.transpose(cv2.imread('/Users/arbues/Documents/DataBasketball/Europe/RegionsIm/RegionsNew/RegionsNew.' + str(iReg).zfill(3) + '.jpeg')[:, :, 0]))
            #imReg = np.fliplr(np.transpose(cv2.imread('/Users/arbues/Documents/DataBasketball/Europe/RegionsIm/RegionsB/' + str(iReg) + '.jpeg')[:, :, 0]))
            for iPlayer in range(0, len(player)):
                for iShot in range(0, len(shotsX[iPlayer])):
                    if shotsX[iPlayer][iShot] != -1 and shotsY[iPlayer][iShot] != -1 and (team[iPlayer][iShot] == targetTeam or targetTeam == ''):
                        if iReg < 8 or iReg == 13:
                            if action[iPlayer][iShot][0] != '3' and imReg[int(shotsX[iPlayer][iShot] - xmin), int(shotsY[iPlayer][iShot] - ymin)] != 0:
                                if action[iPlayer][iShot] in actIn:
                                    shotsIn[iReg - 1, iPlayer] = shotsIn[iReg - 1, iPlayer] + 1
                                else:
                                    shotsOut[iReg - 1, iPlayer] = shotsOut[iReg - 1, iPlayer] + 1
                        else:
                            if action[iPlayer][iShot][0] == '3' and imReg[int(shotsX[iPlayer][iShot] - xmin), int(shotsY[iPlayer][iShot] - ymin)] != 0:
                                if action[iPlayer][iShot] in actIn:
                                    shotsIn[iReg - 1, iPlayer] = shotsIn[iReg - 1, iPlayer] + 1
                                else:
                                    shotsOut[iReg - 1, iPlayer] = shotsOut[iReg - 1, iPlayer] + 1
                        if action[iPlayer][iShot] == 'DUNK' and iReg == 1:
                            layupsIn[0, iPlayer] = layupsIn[0, iPlayer] + 1
                        elif action[iPlayer][iShot][:3] == 'LAY' and iReg == 1:
                            if action[iPlayer][iShot][-1] == 'T':
                                layupsOut[0, iPlayer] = layupsOut[0, iPlayer] + 1
                            else:
                                layupsIn[0, iPlayer] = layupsIn[0, iPlayer] + 1
                    elif shotsX[iPlayer][iShot] == -1 and shotsY[iPlayer][iShot] == -1 and iReg == 1:
                        if action[iPlayer][iShot][:2] == 'FT':
                            if action[iPlayer][iShot][2] == 'M':
                                shotsIn[nRegs, iPlayer] = shotsIn[nRegs, iPlayer] + 1
                            else:
                                shotsOut[nRegs, iPlayer] = shotsOut[nRegs, iPlayer] + 1
        np.save(sFolder + sTypeData + '/shotsInNew' + str(yearBeg) + '-' + str(yearFin) + targetTeam,shotsIn)
        np.save(sFolder + sTypeData + '/shotsOutNew' + str(yearBeg) + '-' + str(yearFin) + targetTeam,shotsOut)
        np.save(sFolder + sTypeData + '/shotsInLAY' + str(yearBeg) + '-' + str(yearFin) + targetTeam,layupsIn)
        np.save(sFolder + sTypeData + '/shotsOutLAY' + str(yearBeg) + '-' + str(yearFin) + targetTeam,layupsOut)
    else:
        shotsIn = np.load(sFolder + sTypeData + '/shotsInNew' + str(yearBeg) + '-' + str(yearFin) + targetTeam + '.npy',allow_pickle=True)
        shotsOut = np.load(sFolder + sTypeData + '/shotsOutNew' + str(yearBeg) + '-' + str(yearFin) + targetTeam + '.npy',allow_pickle=True)
        layupsIn = np.load(sFolder + sTypeData + '/shotsInLAY' + str(yearBeg) + '-' + str(yearFin) + targetTeam + '.npy',allow_pickle=True)
        layupsOut = np.load(sFolder + sTypeData + '/shotsOutLAY' + str(yearBeg) + '-' + str(yearFin) + targetTeam + '.npy',allow_pickle=True)



NameError: name 'cv2' is not defined